# Some stats on Pilot results

In [43]:
import pandas as pd
import json
from scipy import sparse
import pathlib
import pingouin as pg
import krippendorff
import numpy as np
import statsmodels.stats.inter_rater as irr
from scipy.stats import zscore, spearmanr

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

class Colors:
    BLUE = '\033[94m'
    ENDC = '\033[0m'

## Data from pilot 

In [5]:
# Read data from pilot
df = pd.read_json("../../data/results/20240629_145912_parsed_responses.json")[["annotator_id", "cluster_id", "eval_docs", "exemplar_docs", "topic_words"]]

# Construct a dictioanry of dataframes, one for each model type
cluster_dfs_per_model = {model: [] for model in ["ctm", "mallet", "labeled"]}

for model in cluster_dfs_per_model:
    rows = [
        {**doc, 'annotator_id': row['annotator_id'], 'cluster_id': row['cluster_id']}
        for _, row in df.iterrows()
        if model in row['cluster_id']
        for doc in row['eval_docs']
    ]

    flat_df = pd.DataFrame(rows)

    for cluster_id in flat_df['cluster_id'].unique():
        cluster_df = flat_df[flat_df['cluster_id'] == cluster_id]
        
        pivot_df = cluster_df.pivot_table(
            index='annotator_id',
            columns='doc_id',
            values=['prob', 'assigned_to_k', 'fit', 'rank'],
            aggfunc='first'
        )
        
        pivot_df.columns = [f'{doc_id}_{metric}' for metric, doc_id in pivot_df.columns]
        pivot_df.reset_index(inplace=True)
        pivot_df['cluster_id'] = cluster_id
        
        cluster_dfs_per_model[model].append(pivot_df)

## Augmenting with S3

In [47]:
mallet_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/models/mallet")
ctm_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/models/ctm")
corpus_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/train.metadata.jsonl")

s3_mallet = sparse.load_npz(pathlib.Path(mallet_path) / "top_docs_out/S_all.npz").toarray()
s3_ctm = sparse.load_npz(pathlib.Path(ctm_path) / "top_docs_out/S_all.npz").toarray()
s3_mallet.shape

(14290, 50)

In [51]:
def load_model_info(model_path):
    if model_path.stem == "mallet":
        thetas = np.load(model_path / "doctopics.npz.npy")
        betas = np.load(model_path / "beta.npy")
    elif model_path.stem == "ctm":
        thetas = np.load(model_path / "train.theta.npy")
        betas = np.load(model_path / "beta.npy")
    else:
        thetas = sparse.load_npz(pathlib.Path(model_path) / "thetas.npz").toarray()
        betas = np.load(model_path / "betas.npy")
    
    try:
        with open(model_path / "vocab.json") as infile:
            vocab_w2id = json.load(infile)
    except:
        vocab_w2id = {}
        with (pathlib.Path(model_path)/'vocab.txt').open('r', encoding='utf8') as fin:
            for i, line in enumerate(fin):
                wd = line.strip()
                vocab_w2id[wd] = i
    
    try:
        bow = np.load(model_path / "bow_mat.npy")
    except:
        bow = None
    
    vocab_id2w = dict(zip(vocab_w2id.values(), vocab_w2id.keys()))
    keys = [
        [vocab_id2w[idx]
            for idx in row.argsort()[::-1][:15]]
        for row in betas
    ]
        
    return thetas, betas, vocab_w2id, bow, keys

def identify_bad_documents_v2(thetas, betas, documents_texts, vocab_w2id):
    D = len(thetas)
    K = len(betas)
    S3 = np.zeros((D, K))

    for doc in range(D):
        for topic in range(K):
            wd_ids = [vocab_w2id[word]
                      for word in documents_texts[doc] if word in vocab_w2id]
            S3[doc, topic] = np.sum(betas[topic, wd_ids])

    return S3

thetas_mallet, betas_mallet, vocab_w2id_mallet, _, _ = load_model_info(mallet_path)
thetas_ctm, betas_ctm, vocab_w2id_ctm, _, _ = load_model_info(ctm_path)

# Read documents
df = pd.read_json(corpus_path, lines=True)
df["id_top"] = range(len(df))
df["len"] = df["tokenized_text"].apply(lambda x: len(x.split()))
df["text_split"] = df["tokenized_text"].apply(lambda x: x.split())
corpus = df["text_split"].values.tolist()

s3_mallet_v2 = identify_bad_documents_v2(thetas_mallet,betas_mallet,corpus,vocab_w2id_mallet)
s3_ctm_v2 = identify_bad_documents_v2(thetas_ctm,betas_ctm,corpus,vocab_w2id_ctm)

In [62]:
tpc_docs_mallet = {}
for el in cluster_dfs_per_model["mallet"]:
    cols = [col for col in el.columns if "fit" in col]
    tpc_id = int(el["cluster_id"].iloc[0].split("/")[-1])
    docs = [int(col.split("_")[0]) for col in cols]
    s3s = [s3_mallet_v2[doc, tpc_id] for doc in docs]
    tpc_docs_mallet[tpc_id] = s3s
    
tpc_docs_mallet

{20: [0.3688387230801191,
  0.7771553216536251,
  0.45927221106571564,
  0.5054220621840593,
  2.468340142649401,
  0.5335052974170764,
  0.29725088290284607],
 32: [2.2387222438644336,
  1.3618114530580443,
  4.353728087261395,
  2.899006622516556,
  1.839622126996494,
  1.1696299181924426,
  0.203638488507986],
 40: [2.5641399476609177,
  5.177055988074224,
  0.5229485168545768,
  5.675849895772329,
  3.9582311192290835,
  1.462987914504642,
  0.3927233113683926]}

In [63]:
tpc_docs_ctm = {}
for el in cluster_dfs_per_model["ctm"]:
    cols = [col for col in el.columns if "fit" in col]
    tpc_id = int(el["cluster_id"].iloc[0].split("/")[-1])
    docs = [int(col.split("_")[0]) for col in cols]
    s3s = [s3_ctm_v2[doc, tpc_id] for doc in docs]
    tpc_docs_ctm[tpc_id] = s3s
    
tpc_docs_ctm

{34: [0.022406727075576782,
  0.07090145349502563,
  0.2899104654788971,
  0.06289459019899368,
  0.037976257503032684,
  0.032741304486989975,
  0.03475739061832428],
 42: [0.09282709658145905,
  0.08984377235174179,
  0.05710362643003464,
  0.3030792474746704,
  0.02936624549329281,
  0.017437707632780075,
  0.031143471598625183],
 44: [0.35788577795028687,
  0.03792153298854828,
  0.07086992263793945,
  0.12338072061538696,
  0.07340959459543228,
  0.11410756409168243,
  0.08274351805448532]}

## Methods

In [60]:
def calculate_spearman_correlations(df, cols_on, probs):
    spearman_corrs = []
    for _, row in df.iterrows():
        annotator_rank = row[cols_on].values
        spearman_corr, _ = spearmanr(annotator_rank, probs)
        spearman_corrs.append(spearman_corr)
    avg_spearman_corr = sum(spearman_corrs) / len(spearman_corrs)
    return spearman_corrs, avg_spearman_corr

def calculate_kendall_w(data):
    k = data.shape[1]
    rank_means = np.mean(data, axis=0)
    ss_total = np.sum((data - rank_means)**2)
    ss_ranks = np.sum((np.mean(data, axis=1) - np.mean(data))**2) * k
    w = ss_ranks / ss_total
    return w

def calculate_fit_metrics(cluster_df, thr=2):
    
    # Select relevant columns
    df_aux = cluster_df.copy()
    cols_ = ['annotator_id'] + [col for col in df_aux.columns if 'fit' in col]
    df_aux = df_aux[cols_]

    # Melt the dataframe to long format
    df_long = pd.melt(df_aux, id_vars=['annotator_id'], var_name='doc_id', value_name='fit')

    ##################
    # Calculate ICC  #
    ##################
    df_long['item'] = df_long['doc_id'].factorize()[0] + 1 # unique item ID for each unique document
    icc = pg.intraclass_corr(data=df_long, targets='item', raters='annotator_id', ratings='fit')
    # One-way random effects --> ICC1k
    icc = icc.query("Type == 'ICC1k'")['ICC'].values[0]
    print(f"-- -- Intra-class Correlation (ICC1k): {icc:.2f}")

    ########################
    # Krippendorff's Alpha #
    ########################
    fit_matrix = df_long.pivot(index='doc_id', columns='annotator_id', values='fit').to_numpy()
    alpha = krippendorff.alpha(fit_matrix, level_of_measurement='ordinal')
    print(f"-- -- Krippendorff's Alpha: {alpha:.2f}")

    ##################
    # Fleiss' Kappa  #
    ##################
    unique_ratings = np.unique(fit_matrix[~np.isnan(fit_matrix)])
    n_items = fit_matrix.shape[0]
    n_categories = len(unique_ratings)
    table = np.zeros((n_items, n_categories), dtype=int)
    
    rating_to_index = {rating: idx for idx, rating in enumerate(unique_ratings)} # mapping from rating to index
    
    for i, row in enumerate(fit_matrix):
        for rating in row:
            if not np.isnan(rating):
                table[i, rating_to_index[rating]] += 1

    kappa = irr.fleiss_kappa(table, method='fleiss')
    print(f"-- -- Fleiss' Kappa: {kappa:.2f}")

    #########################
    # Ratings Distribution #
    #########################
    unique, counts = np.unique(fit_matrix[~np.isnan(fit_matrix)], return_counts=True)
    print(f"-- -- Ratings Distribution: {dict(zip(unique, counts))}")
    
    ########################
    # Spearman
    ########################
    df_aux = cluster_df.copy()
    fit_columns = [col for col in df_aux.columns if 'fit' in col]
    prob_columns = [col for col in df_aux.columns if 'prob' in col]

    # calculate mean rank vector
    mean_fit_vector = df_aux[fit_columns].mean(axis=0)
    
    # Remove outlisers and recalculate mean rank vector
    diffs = df_aux[fit_columns] - mean_fit_vector
    z_scores = diffs.apply(zscore, axis=0)
    outliers = (z_scores.abs() > thr).any(axis=1)
    df_no_outliers = df_aux[~outliers]
    mean_rank_vector_no_outliers = df_no_outliers[fit_columns].mean(axis=0)

    # Spearman correlation model vs annotators
    spearman_corrs, avg_spearman_corr = calculate_spearman_correlations(df_aux, fit_columns, df_aux[prob_columns].iloc[0].values.tolist())
    print("-- -- Spearman's Fit - Annotators vs Model probs:", spearman_corrs)
    print("-- -- Average Spearman's Fit - Annotators vs Model probs:", avg_spearman_corr)

    # Spearman correlation mean rank after removing outliers vs annotators
    spearman_corrs_no_out, avg_spearman_corr_no_out = calculate_spearman_correlations(df_aux, fit_columns, mean_rank_vector_no_outliers.values.tolist())
    print("-- -- Spearman's FIT - Annotators vs Avg Annotators:", spearman_corrs_no_out)
    print("-- -- Average Spearman's FIT - Annotators vs Avg Annotators:", avg_spearman_corr_no_out)
    
    return icc, alpha, kappa, dict(zip(unique, counts)), avg_spearman_corr, avg_spearman_corr_no_out

def calculate_rank_metrics(df, thr=2):
    
    rank_columns = [col for col in df.columns if 'rank' in col]
    prob_columns = [col for col in df.columns if 'prob' in col]
    
    # calculate mean rank vector
    mean_rank_vector = df[rank_columns].mean(axis=0)
    
    # Remove outlisers and recalculate mean rank vector
    diffs = df[rank_columns] - mean_rank_vector
    z_scores = diffs.apply(zscore, axis=0)
    outliers = (z_scores.abs() > thr).any(axis=1)
    df_no_outliers = df[~outliers]
    mean_rank_vector_no_outliers = df_no_outliers[rank_columns].mean(axis=0)

    # Spearman correlation model vs annotators
    spearman_corrs, avg_spearman_corr = calculate_spearman_correlations(df, rank_columns, df[prob_columns].iloc[0].values.tolist())
    print("-- -- Spearman's Rank - Annotators vs Model probs:", spearman_corrs)
    print("-- -- Average Spearman's - Annotators vs Model probs:", avg_spearman_corr)

    # Spearman correlation mean rank after removing outliers vs annotators
    spearman_corrs_no_out, avg_spearman_corr_no_out = calculate_spearman_correlations(df, rank_columns, mean_rank_vector_no_outliers.values.tolist())
    print("-- -- Spearman's FIT - Annotators vs Avg Annotators:", spearman_corrs_no_out)
    print("-- -- Average Spearman's FIT - Annotators vs Avg Annotators:", avg_spearman_corr_no_out)

    # Kendall's W calculation
    ranks_matrix = df[rank_columns].transpose().to_numpy()
    kendall_w_value = calculate_kendall_w(ranks_matrix)
    print("-- -- Kendall's W:", kendall_w_value)

    return spearman_corrs, avg_spearman_corr, spearman_corrs_no_out, avg_spearman_corr_no_out, kendall_w_value

def calculate_s3_fit_rank_metrics(s3s, df, thr=2):
    
    fit_columns = [col for col in df.columns if 'fit' in col]
    rank_columns = [col for col in df.columns if 'rank' in col]
    tpc = int(df["cluster_id"].iloc[0].split("/")[-1])
    
    #########
    ## FIT #
    #########

    # calculate mean rank vector
    mean_fit_vector = df[fit_columns].mean(axis=0)
    
    # Remove outlisers and recalculate mean rank vector
    diffs = df[fit_columns] - mean_fit_vector
    z_scores = diffs.apply(zscore, axis=0)
    outliers = (z_scores.abs() > thr).any(axis=1)
    df_no_outliers = df[~outliers]
    mean_rank_vector_no_outliers = df_no_outliers[fit_columns].mean(axis=0)

    # Spearman correlation model vs annotators
    spearman_corrs_fit, avg_spearman_corr_fit = calculate_spearman_correlations(df, fit_columns, s3s[tpc])
    print("-- -- Spearman's Fit - Annotators vs Model probs:", spearman_corrs_fit)
    print("-- -- Average Spearman's Fit - Annotators vs Model probs:", avg_spearman_corr_fit)

    # calculate mean rank vector
    mean_rank_vector = df[rank_columns].mean(axis=0)
    
    # Remove outlisers and recalculate mean rank vector
    diffs = df[rank_columns] - mean_rank_vector
    z_scores = diffs.apply(zscore, axis=0)
    outliers = (z_scores.abs() > thr).any(axis=1)
    df_no_outliers = df[~outliers]
    mean_rank_vector_no_outliers = df_no_outliers[rank_columns].mean(axis=0)

    # Spearman rank correlation model vs annotators
    spearman_corrs_rank, avg_spearman_corr_rank = calculate_spearman_correlations(df, rank_columns, s3s[tpc])
    print("-- -- Spearman's Rank - Annotators vs Model probs:", spearman_corrs_rank)
    print("-- -- Average Spearman's Rank - Annotators vs Model probs:", avg_spearman_corr_rank)
    
    return spearman_corrs_fit, avg_spearman_corr_fit, spearman_corrs_rank, avg_spearman_corr_rank

## Stats 

In [98]:
avg_stats = []
std_stats_list = []  # Changed variable name to avoid confusion

for model in cluster_dfs_per_model:
    
    icc_list = []
    alpha_list = []
    kappa_list = []
    ratings_distributions = []
    fit_stats = {
        "Average Spearman's Fit Correlation": [],
        "Average Spearman's Fit Correlation No Outliers": []
    }
    rank_stats = {
        "Average Spearman's Rank Correlation": [],
        "Average Spearman's Rank Correlation No Outliers": [],
        "Kendall's W": []
    }        
    
    this_model_dfs = cluster_dfs_per_model[model]

    for idx, el in enumerate(this_model_dfs):
        print(f"{Colors.BLUE}- --- -- CLUSTER ID: {el['cluster_id'].iloc[0]} -- --- --{Colors.ENDC}")
        
        display(el)

        print(f"{Colors.BLUE}- --- -- FIT STATS -- --- --{Colors.ENDC}")
        icc, alpha, kappa, ratings_distribution, avg_spearman_corr_fit, avg_spearman_corr_no_out_fit = calculate_fit_metrics(el)
        icc_list.append(icc)
        alpha_list.append(alpha)
        kappa_list.append(kappa)
        ratings_distributions.append(ratings_distribution)
        fit_stats["Average Spearman's Fit Correlation"].append(avg_spearman_corr_fit)
        fit_stats["Average Spearman's Fit Correlation No Outliers"].append(avg_spearman_corr_no_out_fit)

        print(f"{Colors.BLUE}- --- -- RANK STATS -- --- --{Colors.ENDC}")
        spearman_correlations, average_spearman_corr, spearman_correlations_no_out, average_spearman_corr_no_out, kendall_w_value = calculate_rank_metrics(el)
        
        rank_stats["Average Spearman's Rank Correlation"].append(average_spearman_corr)
        rank_stats["Average Spearman's Rank Correlation No Outliers"].append(average_spearman_corr_no_out)
        rank_stats["Kendall's W"].append(kendall_w_value)

    average_alpha = np.mean(alpha_list)
    average_kappa = np.mean(kappa_list)

    total_ratings_distribution = {k: sum(d[k] for d in ratings_distributions if k in d) for k in set().union(*ratings_distributions)}
    total_sum = sum(total_ratings_distribution.values())
    normalized_ratings_distribution = {k: v / total_sum for k, v in total_ratings_distribution.items()}
    
    average_rank_stats = {key: np.mean(value) for key, value in rank_stats.items()}
    average_fit_stats = {key: np.mean(value) for key, value in fit_stats.items()}
    
    avg_stats.append([model, average_alpha, average_kappa] + list(average_fit_stats.values()) + list(average_rank_stats.values()))
    
    std_alpha = np.std(alpha_list)
    std_kappa = np.std(kappa_list)

    std_ratings_distribution = {k: np.std([d[k] for d in ratings_distributions if k in d]) for k in total_ratings_distribution.keys()}
    normalized_std_ratings_distribution = {k: v / total_sum for k, v in std_ratings_distribution.items()}


    std_rank_stats = {key: np.std(value) for key, value in rank_stats.items()}
    std_fit_stats = {key: np.std(value) for key, value in fit_stats.items()}
    std_stats_list.append([model, std_alpha, std_kappa] + list(std_fit_stats.values()) + list(std_rank_stats.values()) ) 


- --- -- CLUSTER ID: data/models/ctm/34 -- --- --


,annotator_id,3739_assigned_to_k,4465_assigned_to_k,4764_assigned_to_k,7995_assigned_to_k,9517_assigned_to_k,9531_assigned_to_k,10342_assigned_to_k,3739_fit,4465_fit,4764_fit,7995_fit,9517_fit,9531_fit,10342_fit,3739_prob,4465_prob,4764_prob,7995_prob,9517_prob,9531_prob,10342_prob,3739_rank,4465_rank,4764_rank,7995_rank,9517_rank,9531_rank,10342_rank,cluster_id
0,5b88002a7bb9e90001f0703f,True,True,False,True,True,True,True,3,5,1,5,3,2,3,0.104759,0.314443,0.000027,0.261663,0.157294,0.052344,0.20954,4,1,7,2,5,6,3,data/models/ctm/34
1,6560ae6ce949a9bf95b929f4,True,True,False,True,True,True,True,3,5,1,5,1,2,1,0.104759,0.314443,0.000027,0.261663,0.157294,0.052344,0.20954,3,2,7,1,6,4,5,data/models/ctm/34
2,665a23747dc3dbf8b0cd8da7,True,True,False,True,True,True,True,3,5,1,5,2,3,1,0.104759,0.314443,0.000027,0.261663,0.157294,0.052344,0.20954,6,2,7,1,3,5,4,data/models/ctm/34
3,6679886309dd90c237d80e5d,True,True,False,True,True,True,True,1,5,1,4,1,1,3,0.104759,0.314443,0.000027,0.261663,0.157294,0.052344,0.20954,6,1,5,2,4,7,3,data/models/ctm/34


- --- -- FIT STATS -- --- --
-- -- Intra-class Correlation (ICC1k): 0.94
-- -- Krippendorff's Alpha: -0.07
-- -- Fleiss' Kappa: 0.35
-- -- Ratings Distribution: {1: 10, 2: 3, 3: 7, 4: 1, 5: 7}
Mean fit Vector: 3739_fit     2.50
4465_fit     5.00
4764_fit     1.00
7995_fit     4.75
9517_fit     1.75
9531_fit     2.00
10342_fit    2.00
dtype: float64
-- -- Spearman's Fit - Annotators vs Model probs: [0.9543135154205278, 0.5800729211379678, 0.587378478571482, 0.9063269671749657]
-- -- Average Spearman's Fit - Annotators vs Model probs: 0.7570229705762359
-- -- Spearman's FIT - Annotators vs Avg Annotators: [0.8496617760555312, 0.9251872672604672, 0.8705196492275475, 0.7654211685192612]
-- -- Average Spearman's FIT - Annotators vs Avg Annotators: 0.8526974652657018
- --- -- RANK STATS -- --- --
-- -- Spearman's Rank - Annotators vs Model probs: [-0.9642857142857145, -0.6785714285714287, -0.8928571428571429, -0.8928571428571429]
-- -- Average Spearman's - Annotators vs Model probs: -0.85714

,annotator_id,1494_assigned_to_k,1513_assigned_to_k,1593_assigned_to_k,4764_assigned_to_k,7090_assigned_to_k,9333_assigned_to_k,9399_assigned_to_k,1494_fit,1513_fit,1593_fit,4764_fit,7090_fit,9333_fit,9399_fit,1494_prob,1513_prob,1593_prob,4764_prob,7090_prob,9333_prob,9399_prob,1494_rank,1513_rank,1593_rank,4764_rank,7090_rank,9333_rank,9399_rank,cluster_id
0,5fb87922d40ee25ce27743d7,True,True,True,False,True,False,False,5,5,4,1,3,3,2,0.375484,0.450207,0.300884,0.000046,0.225167,0.150017,0.075012,2,1,4,7,6,3,5,data/models/ctm/42
1,6102f7a4a340c0eec430fbaa,True,True,True,False,True,False,False,5,5,5,1,2,2,1,0.375484,0.450207,0.300884,0.000046,0.225167,0.150017,0.075012,2,1,3,7,5,4,6,data/models/ctm/42
2,662ac63b180632049384144d,True,True,True,False,True,False,False,3,4,3,3,2,4,3,0.375484,0.450207,0.300884,0.000046,0.225167,0.150017,0.075012,5,6,3,7,1,2,4,data/models/ctm/42
3,Jeronimo,True,True,True,False,True,False,False,5,5,5,1,1,2,1,0.375484,0.450207,0.300884,0.000046,0.225167,0.150017,0.075012,3,1,2,7,5,4,6,data/models/ctm/42


- --- -- FIT STATS -- --- --
-- -- Intra-class Correlation (ICC1k): 0.90
-- -- Krippendorff's Alpha: -0.11
-- -- Fleiss' Kappa: 0.12
-- -- Ratings Distribution: {1: 6, 2: 5, 3: 6, 4: 3, 5: 8}
Mean fit Vector: 1494_fit    4.50
1513_fit    4.75
1593_fit    4.25
4764_fit    1.50
7090_fit    2.00
9333_fit    2.75
9399_fit    1.75
dtype: float64
-- -- Spearman's Fit - Annotators vs Model probs: [0.981980506061966, 0.9449111825230682, 0.23904572186687872, 0.8486684247915056]
-- -- Average Spearman's Fit - Annotators vs Model probs: 0.7536514588108546
-- -- Spearman's FIT - Annotators vs Avg Annotators: [0.981980506061966, 0.9449111825230682, 0.45817096691151754, 0.9258200997725515]
-- -- Average Spearman's FIT - Annotators vs Avg Annotators: 0.8277206888172758
- --- -- RANK STATS -- --- --
-- -- Spearman's Rank - Annotators vs Model probs: [-0.8214285714285715, -0.9642857142857145, 0.0, -0.9285714285714288]
-- -- Average Spearman's - Annotators vs Model probs: -0.6785714285714287
-- -- Spear

...

- --- -- FIT STATS -- --- --
-- -- Intra-class Correlation (ICC1k): 0.84
-- -- Krippendorff's Alpha: -0.00
-- -- Fleiss' Kappa: 0.38
-- -- Ratings Distribution: {1: 16, 2: 3, 3: 2, 4: 1, 5: 6}
Mean fit Vector: 1420_fit     1.0
1534_fit     5.0
1564_fit     3.0
5209_fit     1.5
7337_fit     1.0
10148_fit    2.0
10851_fit    2.0
dtype: float64
-- -- Spearman's Fit - Annotators vs Model probs: [0.8017837257372733, 0.8964214570007953, 0.8017837257372733, 0.6910233190806426]
-- -- Average Spearman's Fit - Annotators vs Model probs: 0.7977530568889962
-- -- Spearman's FIT - Annotators vs Avg Annotators: [0.8164965809277261, 0.8621558775544281, 0.8164965809277261, 0.7314814814814816]
-- -- Average Spearman's FIT - Annotators vs Avg Annotators: 0.8066576302228405
- --- -- RANK STATS -- --- --
-- -- Spearman's Rank - Annotators vs Model probs: [-0.8928571428571429, -0.9642857142857145, -1.0, -0.7500000000000002]
-- -- Average Spearman's - Annotators vs Model probs: -0.9017857142857144
-- -- Spe

,annotator_id,1084_assigned_to_k,2251_assigned_to_k,2391_assigned_to_k,2433_assigned_to_k,3263_assigned_to_k,3957_assigned_to_k,11959_assigned_to_k,1084_fit,2251_fit,2391_fit,2433_fit,3263_fit,3957_fit,11959_fit,1084_prob,2251_prob,2391_prob,2433_prob,3263_prob,3957_prob,11959_prob,1084_rank,2251_rank,2391_rank,2433_rank,3263_rank,3957_rank,11959_rank,cluster_id
0,5f3430f7b2ea6115cb8564bb,False,True,True,False,False,False,False,1,5,4,1,3,1,1,0.111633,1.669551,1.558127,-0.000009,0.334768,0.223172,0.446349,7,1,2,5,3,6,4,data/models/labeled/category-45/27
1,5f416fceecc86b11a6f7ec1e,False,True,True,False,False,False,False,1,5,5,1,4,1,3,0.111633,1.669551,1.558127,-0.000009,0.334768,0.223172,0.446349,7,1,2,6,3,5,4,data/models/labeled/category-45/27
2,5fcdbe153ce79b287e15344e,False,True,True,False,False,False,False,1,5,5,1,3,1,2,0.111633,1.669551,1.558127,-0.000009,0.334768,0.223172,0.446349,6,1,2,7,3,5,4,data/models/labeled/category-45/27
3,663e45f8e326d67a176124d0,False,True,True,False,False,False,False,1,5,5,1,2,5,1,0.111633,1.669551,1.558127,-0.000009,0.334768,0.223172,0.446349,7,2,1,6,3,5,4,data/models/labeled/category-45/27
4,6658b2050980b63fc8fdbd13,False,True,True,False,False,False,False,1,5,5,1,3,1,3,0.111633,1.669551,1.558127,-0.000009,0.334768,0.223172,0.446349,5,1,2,7,3,6,4,data/models/labeled/category-45/27


- --- -- FIT STATS -- --- --
-- -- Intra-class Correlation (ICC1k): 0.95
-- -- Krippendorff's Alpha: -0.11
-- -- Fleiss' Kappa: 0.52
-- -- Ratings Distribution: {1: 16, 2: 2, 3: 5, 4: 2, 5: 10}
Mean fit Vector: 1084_fit     1.0
2251_fit     5.0
2391_fit     4.8
2433_fit     1.0
3263_fit     3.0
3957_fit     1.8
11959_fit    2.0
dtype: float64
-- -- Spearman's Fit - Annotators vs Model probs: [0.8078131663950782, 0.9168894559922718, 0.9168894559922718, 0.6172133998483676, 0.9449111825230682]
-- -- Average Spearman's Fit - Annotators vs Model probs: 0.8407433321502115
-- -- Spearman's FIT - Annotators vs Avg Annotators: [0.9145291883606756, 0.9629500128629354, 0.9629500128629354, 0.7006490497453707, 0.9534625892455925]
-- -- Average Spearman's FIT - Annotators vs Avg Annotators: 0.898908170615502
- --- -- RANK STATS -- --- --
-- -- Spearman's Rank - Annotators vs Model probs: [-0.8571428571428573, -0.9285714285714288, -0.9642857142857145, -0.8928571428571429, -0.9285714285714288]
-- -- A

,annotator_id,2247_assigned_to_k,5493_assigned_to_k,6608_assigned_to_k,10244_assigned_to_k,10554_assigned_to_k,10654_assigned_to_k,10728_assigned_to_k,2247_fit,5493_fit,6608_fit,10244_fit,10554_fit,10654_fit,10728_fit,2247_prob,5493_prob,6608_prob,10244_prob,10554_prob,10654_prob,10728_prob,2247_rank,5493_rank,6608_rank,10244_rank,10554_rank,10654_rank,10728_rank,cluster_id
0,5ff4ebaadd8cf1c332708eea,False,False,False,False,True,True,True,1,1,1,1,4,4,5,0.000005,0.261404,0.130623,0.386698,1.523232,1.653239,1.783802,6,4,7,5,3,2,1,data/models/labeled/category-45/34
1,614f7300717876eb9b5601b0,False,False,False,False,True,True,True,1,1,1,1,3,3,5,0.000005,0.261404,0.130623,0.386698,1.523232,1.653239,1.783802,5,6,7,4,3,2,1,data/models/labeled/category-45/34
2,64135dd9d2ef675b8e1799ef,False,False,False,False,True,True,True,1,1,1,1,3,3,5,0.000005,0.261404,0.130623,0.386698,1.523232,1.653239,1.783802,6,5,7,4,2,3,1,data/models/labeled/category-45/34
3,657eff2f38428b854dc1296b,False,False,False,False,True,True,True,2,1,2,1,2,4,5,0.000005,0.261404,0.130623,0.386698,1.523232,1.653239,1.783802,5,4,7,6,1,3,2,data/models/labeled/category-45/34
4,66589cb40bba8a88da3c3a88,False,False,False,False,True,True,True,1,1,1,1,5,5,5,0.000005,0.261404,0.130623,0.386698,1.523232,1.653239,1.783802,6,5,7,4,3,2,1,data/models/labeled/category-45/34


- --- -- FIT STATS -- --- --
-- -- Intra-class Correlation (ICC1k): 0.98
-- -- Krippendorff's Alpha: -0.11
-- -- Fleiss' Kappa: 0.47
-- -- Ratings Distribution: {1: 18, 2: 3, 3: 4, 4: 3, 5: 7}
Mean fit Vector: 2247_fit     1.2
5493_fit     1.0
6608_fit     1.2
10244_fit    1.0
10554_fit    3.4
10654_fit    3.8
10728_fit    5.0
dtype: float64
-- -- Spearman's Fit - Annotators vs Model probs: [0.8964214570007953, 0.8964214570007953, 0.8964214570007953, 0.5800729211379678, 0.8660254037844388]
-- -- Average Spearman's Fit - Annotators vs Model probs: 0.8270725391849585
-- -- Spearman's FIT - Annotators vs Avg Annotators: [0.9128709291752769, 0.9128709291752769, 0.9128709291752769, 0.9718253158075503, 0.881917103688197]
-- -- Average Spearman's FIT - Annotators vs Avg Annotators: 0.9184710414043156
- --- -- RANK STATS -- --- --
-- -- Spearman's Rank - Annotators vs Model probs: [-0.9285714285714288, -0.8928571428571429, -0.9285714285714288, -0.7142857142857144, -0.9642857142857145]
-- -- Av

In [99]:
print("MEAN STATS")
avg_stats_df = pd.DataFrame(avg_stats, columns=["Model", "Fit Krippendorff's Alpha", "Fit Fleiss' Kappa", "Spearman's Fit Model vs Annotators", "Spearman's Fit No Outliers vs Annotators", "Spearman's Rank Model vs Annotators", "Spearman's Rank No Outliers vs Annotators", "Rank Kendall's W"])

display(avg_stats_df)

print("STD STATS")

std_stats_df = pd.DataFrame(std_stats_list, columns=["Model", "Fit Krippendorff's Alpha", "Fit Fleiss' Kappa", "Spearman's Fit Model vs Annotators", "Spearman's Fit No Outliers vs Annotators", "Spearman's Rank Model vs Annotators", "Spearman's Rank No Outliers vs Annotators", "Rank Kendall's W"])

display(std_stats_df)

MEAN STATS


,Model,Fit Krippendorff's Alpha,Fit Fleiss' Kappa,Spearman's Fit Model vs Annotators,Spearman's Fit No Outliers vs Annotators,Spearman's Rank Model vs Annotators,Spearman's Rank No Outliers vs Annotators,Rank Kendall's W
0,ctm,-0.056493,0.199410,0.789476,0.853476,-0.806548,0.838074,0.747024
1,mallet,0.058496,0.208711,0.601592,0.792005,-0.630952,0.883484,0.816964
2,labeled,-0.075894,0.452958,0.821856,0.874679,-0.900595,0.934370,0.893033


STD STATS


,Model,Fit Krippendorff's Alpha,Fit Fleiss' Kappa,Spearman's Fit Model vs Annotators,Spearman's Fit No Outliers vs Annotators,Spearman's Rank Model vs Annotators,Spearman's Rank No Outliers vs Annotators,Rank Kendall's W
0,ctm,0.047997,0.107865,0.048299,0.021353,0.091151,0.086937,0.124485
1,mallet,0.075174,0.139611,0.179833,0.026209,0.233095,0.057219,0.089062
2,labeled,0.053627,0.059161,0.017934,0.048757,0.011695,0.023641,0.045427


### Betas-based

In [64]:
for model in cluster_dfs_per_model:
    
    icc_list = []
    alpha_list = []
    kappa_list = []
    ratings_distributions = []
    fit_stats = {
        "Average Spearman's Fit Correlation": [],
        "Average Spearman's Fit Correlation No Outliers": []
    }
    rank_stats = {
        "Average Spearman's Rank Correlation": [],
        "Average Spearman's Rank Correlation No Outliers": [],
        "Kendall's W": []
    }        
    
    this_model_dfs = cluster_dfs_per_model[model]

    for idx, el in enumerate(this_model_dfs):
        
        if model == "ctm":
            s3s = tpc_docs_ctm
        elif model == "mallet":
            s3s = tpc_docs_mallet
        else:
            print("Not supported yet for baseline")
        
        if model == "ctm" or model == "mallet":
            spearman_corrs_fit, avg_spearman_corr_fit, spearman_corrs_rank, avg_spearman_corr_rank = calculate_s3_fit_rank_metrics(s3s, el)

-- -- Spearman's Fit - Annotators vs Model probs: [0.09356014857063998, 0.018712029714127994, 0.0, 0.39405520311955033]
-- -- Average Spearman's Fit - Annotators vs Model probs: 0.12658184535107958
-- -- Spearman's Rank - Annotators vs Model probs: [-0.07142857142857144, 0.14285714285714288, -0.21428571428571433, -0.5714285714285715]
-- -- Average Spearman's Rank - Annotators vs Model probs: -0.1785714285714286
-- -- Spearman's Fit - Annotators vs Model probs: [0.10910894511799621, 0.13228756555322954, -0.03984095364447979, 0.23145502494313788]
-- -- Average Spearman's Fit - Annotators vs Model probs: 0.10825264549247096
-- -- Spearman's Rank - Annotators vs Model probs: [0.0, -0.07142857142857144, 0.8928571428571429, 0.0]
-- -- Average Spearman's Rank - Annotators vs Model probs: 0.20535714285714288
-- -- Spearman's Fit - Annotators vs Model probs: [0.17732484140379767, -0.3555285645684319, -0.4330127018922194, -0.23904572186687872]
-- -- Average Spearman's Fit - Annotators vs Model p